In [ ]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

/home/tushar/work/LLM/vector_db_practise/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
qdrant = QdrantClient(url="http://localhost:6333")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

create collection in Qdrant vector DB

In [ ]:
collection = "practise_collection"
# create collection only if it does not exist
if not qdrant.collection_exists(collection_name=collection):
    qdrant.create_collection(
        collection_name=collection,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE),
    )

For a given data, create its embeddings and store it into Qdrant vector DB

In [ ]:
docs = [
    "Dogs are loyal",
    "Cats are independent",
    "Puppies are young dogs",
]

# create embeddings for the documents
embeddings = embedder.encode(docs)

In [ ]:
# embedding should be saved in Qdrant vector DB collection in the form of points
# prepare points to be inserted
points= []
for i, (txt, emb) in enumerate(zip(docs, embeddings)):
    points.append(PointStruct(id=i, vector=emb.tolist(), payload={"text": txt}))

In [32]:
qdrant.upsert(collection_name=collection, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

Now search similar document from Vector DB 

In [38]:
query = "Tell me about dogs"
query_embedding = embedder.encode([query])[0]
search_results = qdrant.search(
    collection_name=collection,
    query_vector=query_embedding.tolist(),
    limit=2,
)

/tmp/ipykernel_58431/3926308429.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant.search(


In [39]:
search_results

[ScoredPoint(id=0, version=0, score=0.57007104, payload={'text': 'Dogs are loyal'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2, version=0, score=0.54749566, payload={'text': 'Puppies are young dogs'}, vector=None, shard_key=None, order_value=None)]

In [40]:
for search_result in search_results:
    print(f"Score: {search_result.score}, Text: {search_result.payload['text']}")

Score: 0.57007104, Text: Dogs are loyal
Score: 0.54749566, Text: Puppies are young dogs
